In [31]:
import numpy as np
import pandas as pd
import networkx as nx
import re

## Read the book metadata dataset

In [2]:
df = pd.read_parquet('metadata_en.parquet')
df

,title,creator,birthdate,deathdate,publisher,issued,rights,language,subjects,description,Index
1,The Master Spirit,"Magnay, William, Sir",1855,1917,Project Gutenberg,2022-03-25,Public domain in the USA.,en,"[London (England) -- Fiction, Love stories, Ad...",[Reading ease score: 71.3 (7th grade). Fairly ...,67703
2,The Family on Wheels,"Oxley, J. Macdonald (James Macdonald)",1855,1907,Project Gutenberg,2017-11-05,Public domain in the USA.,en,"[Orphans -- Juvenile fiction, Siblings -- Juve...",[Reading ease score: 77.9 (7th grade). Fairly ...,55891
3,Three Good Giants\nWhose Ancient Deeds are rec...,"Rabelais, François",1835,1901,Project Gutenberg,2019-04-09,Public domain in the USA.,en,"[Fantasy fiction, Conduct of life -- Juvenile ...",[Reading ease score: 76.2 (7th grade). Fairly ...,59235
4,Drake's Road Book of the Grand Junction Railwa...,"Drake, James, active 1825",None,None,Project Gutenberg,2013-07-31,Public domain in the USA.,en,[Grand Junction Railway Company (Great Britain...,[Reading ease score: 65.5 (8th & 9th grade). N...,43367
5,Facts and Speculations on the Origin and Histo...,"Chatto, William Andrew",1799,1864,Project Gutenberg,2014-05-04,Public domain in the USA.,en,"[Playing cards -- History, GV]",[Reading ease score: 62.7 (8th & 9th grade). N...,45584
...,...,...,...,...,...,...,...,...,...,...,...
74677,Reform and Politics\r\nPart 2 from The Works o...,"Whittier, John Greenleaf",1807,1892,Project Gutenberg,2005-12-01,Public domain in the USA.,en,"[United States -- Politics and government, PS]",[Reading ease score: 49.7 (College-level). Dif...,9596
74678,"The Speedwell Boys and Their Ice Racer; Or, Lo...","Rockwood, Roy",None,None,Project Gutenberg,2015-06-07,Public domain in the USA.,en,"[Iceboating -- Juvenile fiction, PZ]",[Reading ease score: 83.7 (6th grade). Easy to...,49162
74680,The Starling: A Scottish Story,"Macleod, Norman",1812,1872,Project Gutenberg,2013-02-12,Public domain in the USA.,en,"[Clergy -- Fiction, Villages -- Fiction, Scotl...",[Reading ease score: 78.4 (7th grade). Fairly ...,41989
74682,"Chronicles of England, Scotland and Ireland (2...","Holinshed, Raphael",None,None,Project Gutenberg,2005-09-25,Public domain in the USA.,en,"[Great Britain -- History -- Tudors, 1485-1603...",[Reading ease score: 58.5 (10th to 12th grade)...,16749


Basic information about the dataset

In [13]:
print(f"Dataframe's shape: {df.shape}\n")
print(df.info())

Dataframe's shape: (59852, 11)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59852 entries, 1 to 74683
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        59847 non-null  object
 1   creator      59698 non-null  object
 2   birthdate    49715 non-null  object
 3   deathdate    48613 non-null  object
 4   publisher    59852 non-null  object
 5   issued       59852 non-null  object
 6   rights       59852 non-null  object
 7   language     59852 non-null  object
 8   subjects     59852 non-null  object
 9   description  59852 non-null  object
 10  Index        59852 non-null  object
dtypes: object(11)
memory usage: 5.5+ MB
None


## Data Preprocessing

In [16]:
print("Birthdate's unique values:\n", df['birthdate'].unique())
print("Deathdate's unique values:\n", df['deathdate'].unique())

Birthdate's unique values:
 ['1855' '1835' None '1799' '1836' '1873' '1804' '1803' '1837' '1911'
 '1859' '1812' '1865' '1813' '1851' '1849' '1862' '1879' '1846' '1860'
 '1885' '1872' '1822' '1910' '1814' '1876' '1893' '1903' '1882' '1852'
 '1785' '1838' '1877' '1834' '1824' '1870' '1861' '1802' '1868' '1842'
 '1892' '1858' '1664' '1866' '1880' '1867' '1881' '1854' '1841' '1839'
 '1856' '1848' '1853' '1875' '1863' '1828' '1871' '1772' '1923' '1830'
 '1751' '1889' '1794' '1831' '1899' '1878' '1925' '1922' '1753' '1832'
 '1883' '1633' '1817' '1907' '1811' '1845' '1869' '1747' '1850' '1971'
 '1833' '1717' '1914' '1826' '1840' '1930' '1791' '1810' '1801' '1844'
 '1564' '1928' '1857' '1792' '1901' '1787' '1703' '1821' '1888' '1827'
 '1819' '1890' '1805' '1809' '1843' '1797' '1783' '1820' '1776' '1641'
 '1511' '1864' '1912' '1823' '1886' '1927' '1847' '1789' '1469' '1806'
 '1755' '1497' '1829' '1894' '1760' '1897' '1775' '-431' '1527' '1896'
 '1807' '1796' '1887' '1771' '1728' '1918' '1825' '

In [24]:
df['birthdate'] = df['birthdate'].astype('Int64')
df['deathdate'] = df['deathdate'].astype('Int64')

print(df.info())

display(df[['birthdate', 'deathdate']].head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59852 entries, 1 to 74683
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        59847 non-null  object
 1   creator      59698 non-null  object
 2   birthdate    49715 non-null  Int64 
 3   deathdate    48613 non-null  Int64 
 4   publisher    59852 non-null  object
 5   issued       59852 non-null  object
 6   rights       59852 non-null  object
 7   language     59852 non-null  object
 8   subjects     59852 non-null  object
 9   description  59852 non-null  object
 10  Index        59852 non-null  object
dtypes: Int64(2), object(9)
memory usage: 5.6+ MB
None


,birthdate,deathdate
1,1855,1917
2,1855,1907
3,1835,1901
4,<NA>,<NA>
5,1799,1864


In [27]:
print("Number of unique values in the language column: {0}".format(
    df['language'].unique().shape[0]
))

Number of unique values in the language column: 1


In [29]:
if 'language' in df.columns:
    del df['language']

display(df.head())

,title,creator,birthdate,deathdate,publisher,issued,rights,subjects,description,Index
1,The Master Spirit,"Magnay, William, Sir",1855,1917,Project Gutenberg,2022-03-25,Public domain in the USA.,"[London (England) -- Fiction, Love stories, Ad...",[Reading ease score: 71.3 (7th grade). Fairly ...,67703
2,The Family on Wheels,"Oxley, J. Macdonald (James Macdonald)",1855,1907,Project Gutenberg,2017-11-05,Public domain in the USA.,"[Orphans -- Juvenile fiction, Siblings -- Juve...",[Reading ease score: 77.9 (7th grade). Fairly ...,55891
3,Three Good Giants\nWhose Ancient Deeds are rec...,"Rabelais, François",1835,1901,Project Gutenberg,2019-04-09,Public domain in the USA.,"[Fantasy fiction, Conduct of life -- Juvenile ...",[Reading ease score: 76.2 (7th grade). Fairly ...,59235
4,Drake's Road Book of the Grand Junction Railwa...,"Drake, James, active 1825",<NA>,<NA>,Project Gutenberg,2013-07-31,Public domain in the USA.,[Grand Junction Railway Company (Great Britain...,[Reading ease score: 65.5 (8th & 9th grade). N...,43367
5,Facts and Speculations on the Origin and Histo...,"Chatto, William Andrew",1799,1864,Project Gutenberg,2014-05-04,Public domain in the USA.,"[Playing cards -- History, GV]",[Reading ease score: 62.7 (8th & 9th grade). N...,45584


In [39]:
reading_ease_scores = list()
reading_ease_levels = list()
reading_ease_comments = list()

for i, descriptions in enumerate(df['description']):
    for description in descriptions:
        if re.search("Reading ease score:", description):
            match = re.findall("Reading ease score: ([\d\.]+)", description)
            reading_ease_scores.append(float(match[0]))

            match = re.findall("Reading ease score:\s[\d\.]+\s\((.+)\)", description)
            reading_ease_levels.append(match[0])

            match = re.findall("Reading ease score:\s[\d\.]+\s\(.+\)")


['7th grade']
['7th grade']
['7th grade']
['8th & 9th grade']
['8th & 9th grade']
['7th grade']
['6th grade']
['6th grade']
['8th & 9th grade']
['8th & 9th grade']
['College-level']
['7th grade']
['7th grade']
['6th grade']
['6th grade']
['10th to 12th grade']
['7th grade']
['College-level']
['8th & 9th grade']
['8th & 9th grade']
['7th grade']
['8th & 9th grade']
['7th grade']
['6th grade']
['7th grade']
['8th & 9th grade']
['6th grade']
['6th grade']
['8th & 9th grade']
['7th grade']
['8th & 9th grade']
['College-level']
['6th grade']
['7th grade']
['8th & 9th grade']
['College-level']
['8th & 9th grade']
['College-level']
['College-level']
['6th grade']
['10th to 12th grade']
['7th grade']
['8th & 9th grade']
['6th grade']
['7th grade']
['10th to 12th grade']
['6th grade']
['7th grade']
['10th to 12th grade']
['8th & 9th grade']
['8th & 9th grade']
['8th & 9th grade']
['7th grade']
['5th grade']
['7th grade']
['10th to 12th grade']
['10th to 12th grade']
['College-level']
['College-